In [1]:
%load_ext autoreload
%autoreload 2

## Generating the inference script

In [15]:
# Import necessary modules from LangChain
from langchain.chat_models import ChatOpenAI
from talk_openai import MyOpenAI
from langchain.schema import HumanMessage
import os
from utils import *
import re

In [3]:
MODEL_NAME = "gpt-4o-mini"
# Create a ChatOpenAI object 
inference_script_generator_llm = MyOpenAI(model=MODEL_NAME)
inference_script_validator_llm = MyOpenAI(model=MODEL_NAME)

/Users/neel/Developer/deploy_wizard/talk_openai.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(model=model, temperature=0)


In [4]:
def get_file_path():
    """Prompt the user for the training Python file path."""
    file_path = input("Enter the path to the training Python file: ").strip()
    return file_path

def validate_file(file_path):
    """Validate if the file exists and is a Python script."""
    if not os.path.exists(file_path):
        raise FileNotFoundError("File not found. Please provide a valid path.")
    
    if not file_path.endswith('.py'):
        raise ValueError("Unsupported file format. Only Python (.py) files are allowed.")
    
    return True

In [5]:
def get_user_inputs():
    """Ask the user to provide model details and data shape."""
    print("To determine the input shape of a single record, you can do the following:")
    print("1. If you have a Pandas DataFrame, use df.iloc[0].shape to get the shape of a single row.")
    print("2. If using NumPy, call data[0].shape on your dataset.")
    print("3. If your data is in a list format, check the length of the first element using len(data[0]).")
    
    model_type = input("Enter the model type (Binary Classification, Multiclass Classification, Regression): ").strip()
    data_shape = input("Enter the expected shape of a single row of input data (e.g., (1, 10) for 10 features): ").strip()
    return model_type, data_shape



# model_type, data_shape = get_user_inputs()
model_type = 'Multiclass Classification'
data_shape = '(, 4)'
# input_file = get_file_path()
input_file = '/Users/neel/Developer/deploy_wizard/iris_model_training/training.py'
training_script = file_reader(input_file)

In [24]:
def generate_inference_script_with_llm(training_script, model_type, data_shape, feedback=None):
    """Use an LLM to generate an inference script based on the provided training script and details."""
    if feedback is None:        
        #if feedback is None, then we are generating the initial prompt
        messages = [
            {"role": "system", "content": "You are an expert Python developer."},
            {"role": "user", "content": f'''
            You are an expert Python developer. Your task is to generate an inference script that exposes a trained machine learning model as an API.

            ### Role: 
            You are to assist in writing an inference script for an AI agent that will deploy a trained model.

            ### Context:
            - The training script provided is below:
            """
            {training_script}
            """
            - The model type is {model_type}.
            - The expected input shape for predictions is {data_shape}.

            ### Action:
            Write a Python script that:
            - Loads the trained model.
            - Accepts input data via an API.
            - Preprocesses the input.
            - Makes predictions using the model.
            - Returns the prediction as JSON.
            - Uses FastAPI for API exposure.

            ### Expected Output Format:
            A complete, standalone Python script that follows best practices and is ready for execution.
            '''}
            ]
    else:
        #feedback is present so we treat it as a continuation of the conversation
        messages= feedback
    response = inference_script_generator_llm.invoke(messages)
    return response

In [29]:
def evaluate_inference_script_with_llm(inference_script):
    """Use an LLM to evaluate the generated inference script and provide feedback."""
    messages = [
        {"role": "system", "content": "You are a code reviewer and expert in Python API development."},
        {"role": "user", "content": f"""
        You are reviewing an inference script generated for exposing a trained ML model as an API.

        ### Task:
        - Analyze the script for correctness, best practices, and completeness.
        - Identify any missing components or improvements.
        - If the script is perfect, respond with 'No changes needed.'
        - If changes are required, specify what needs to be improved.

        ### Script to review:
        ```
        {inference_script}
        ```

        ### Expected Output:
        - Either 'No changes needed.' OR a detailed improvement plan.
        """}
    ]
    
    return inference_script_validator_llm.invoke(messages)

def actor_critic_inference_script(training_script, model_type, data_shape):
    """
    Runs the actor-critic loop:
    - Actor generates an inference script.
    - Critic evaluates the script.
    - If the script is sufficient, exit early.
    - Otherwise, Actor refines the script based on feedback.
    - Max iterations: 2
    """
    feedback = None  # No feedback initially

    for _ in range(1):  # Max iterations: 1
        # Step 1: Actor generates inference script
        inference_script = generate_inference_script_with_llm(training_script, model_type, data_shape, feedback)

        # Step 2: Critic evaluates the script
        critic_feedback = evaluate_inference_script_with_llm(inference_script)

        if "no changes needed" in critic_feedback.lower():
            print("✅ Inference script is satisfactory. Exiting early.")
            return inference_script  # Early exit if script is sufficient

        # Step 3: Actor refines script using critic's feedback
        feedback = [
            {"role": "assistant", "content": inference_script},
            {"role": "user", "content": f"Revise based on this feedback:\n{critic_feedback}"}
        ]

    print("🔄 Max iterations reached. Returning final script.")
    return inference_script

In [30]:
raw_inference_script = actor_critic_inference_script(training_script, model_type, data_shape)

🔄 Max iterations reached. Returning final script.


In [32]:
print(raw_inference_script)

Here's a complete Python script that uses FastAPI to expose the trained RandomForestClassifier model as an API. This script will load the model, accept input data, preprocess it, make predictions, and return the results in JSON format.

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np

# Load the trained model
model = joblib.load('iris_model.pkl')

# Create a FastAPI instance
app = FastAPI()

# Define the input data model
class IrisInput(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

# Define the prediction endpoint
@app.post("/predict")
def predict(iris_input: IrisInput):
    # Prepare the input data for prediction
    input_data = np.array([[iris_input.sepal_length, iris_input.sepal_width,
                             iris_input.petal_length, iris_input.petal_width]])
    
    # Make prediction
    try:
        prediction = model.predict(input_data)
        

In [33]:
import re

def extract_python_script(llm_response):
    """
    Extracts the main Python script from an LLM response.

    Args:
        llm_response (str): The response text containing a Python code block.

    Returns:
        str: The extracted Python script, or an empty string if no script is found.
    """
    match = re.search(r"```python\n(.*?)\n```", llm_response, re.DOTALL)
    return match.group(1) if match else ""


inference_script = extract_python_script(raw_inference_script)

In [34]:
print(inference_script)

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np

# Load the trained model
model = joblib.load('iris_model.pkl')

# Create a FastAPI instance
app = FastAPI()

# Define the input data model
class IrisInput(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

# Define the prediction endpoint
@app.post("/predict")
def predict(iris_input: IrisInput):
    # Prepare the input data for prediction
    input_data = np.array([[iris_input.sepal_length, iris_input.sepal_width,
                             iris_input.petal_length, iris_input.petal_width]])
    
    # Make prediction
    try:
        prediction = model.predict(input_data)
        predicted_class = int(prediction[0])  # Get the predicted class as an integer
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    
    # Return the prediction as JSON
    return {"predicted_class": predicted_

In [36]:
from utils import *

In [ ]:
#saving the inference script to a file
inference_script_path = '/Users/neel/Developer/deploy_wizard/iris_model_inference/inference.py'
write_to_file(inference_script_path, inference_script)


## Generating requirements.txt

In [40]:
import re

def extract_imported_libraries(python_script):
    """
    Extracts imported libraries from a Python script.

    Args:
        python_script (str): The Python script content.

    Returns:
        list: A list of unique libraries found in the script.
    """
    matches = re.findall(r"^\s*(?:import|from)\s+([\w\d_\.]+)", python_script, re.MULTILINE)
    return list(set(matches))  # Remove duplicates


def generate_requirements_with_llm(inference_script):
    """
    Uses an LLM to generate a requirements.txt file based on the inference script.

    Args:
        inference_script (str): The Python inference script.

    Returns:
        str: The generated requirements.txt content.
    """
    imported_libraries = extract_imported_libraries(inference_script)
    libraries_str = ", ".join(imported_libraries)

    messages = [
        {"role": "system", "content": "You are an expert Python package manager."},
        {"role": "user", "content": f"""
        Given the following imported libraries: {libraries_str}

        ### Task:
        - Identify the necessary dependencies and versions.
        - Generate a `requirements.txt` file.
        - Ensure compatibility based on best practices.

        ### Expected Output Format:
        ```
        package_name==X.Y.Z
        package_name_2>=A.B.C
        package_name_3
        ```
        - Each package should be on a new line.
        - Include version constraints where necessary.
        - Ensure that the packages are installable via pip.
        """}
    ]

    return requirements_generator_llm.invoke(messages)

In [41]:
# Create a new LLM instance for generating requirements.txt
requirements_generator_llm = MyOpenAI(model=MODEL_NAME)

# Extract Python script from LLM response
extracted_script = extract_python_script(raw_inference_script)

# Generate the requirements.txt content
requirements_txt = generate_requirements_with_llm(extracted_script)
print(requirements_txt)

Based on the libraries you've imported, here are the necessary dependencies and their recommended versions as of October 2023. I've ensured compatibility and best practices for each package.

### Recommended Versions:
- **pydantic**: The latest stable version is `1.10.9`. Pydantic is a data validation and settings management library that is widely used with FastAPI.
- **fastapi**: The latest stable version is `0.95.2`. FastAPI is a modern web framework for building APIs with Python 3.6+ based on standard Python type hints.
- **joblib**: The latest stable version is `1.3.2`. Joblib is used for lightweight pipelining in Python and is often used for saving and loading models.
- **numpy**: The latest stable version is `1.24.3`. NumPy is a fundamental package for scientific computing with Python.
- **uvicorn**: The latest stable version is `0.22.0`. Uvicorn is a lightning-fast ASGI server implementation, using `uvloop` and `httptools`.

### Generated `requirements.txt`:
```
pydantic==1.10.9

In [42]:
def extract_requirements_txt(llm_response):
    """
    Extracts the main contents of the `requirements.txt` file from an LLM response.

    Args:
        llm_response (str): The response text containing the `requirements.txt` section.

    Returns:
        str: The extracted `requirements.txt` content as a string.
    """
    match = re.search(r"```(?:\w+\n)?(.*?)\n```", llm_response, re.DOTALL)
    return match.group(1).strip() if match else ""

In [43]:
requirements_txt_content = extract_requirements_txt(requirements_txt)
print(requirements_txt_content)

pydantic==1.10.9
fastapi==0.95.2
joblib==1.3.2
numpy==1.24.3
uvicorn==0.22.0


In [44]:
#saving the requirements.txt to a file
requirements_txt_path = '/Users/neel/Developer/deploy_wizard/iris_model_training/requirements.txt'
write_to_file(requirements_txt_path, requirements_txt_content)

## Containerization 

In [45]:
def modify_model_loading_with_llm(inference_script):
    """
    Uses an LLM to modify the inference script, ensuring the model is loaded from an ENV variable if set,
    while keeping the rest of the script unchanged.

    Args:
        inference_script (str): The raw inference script.

    Returns:
        str: The modified inference script with the environment-based model loading.
    """
    messages = [
        {"role": "system", "content": "You are an expert Python developer."},
        {"role": "user", "content": f"""
        The following Python script is an inference API.

        ### Task:
        - Identify the line(s) where the model is loaded (e.g., `model = joblib.load("model.pkl")`).
        - Modify only those lines to:
          - Load the model from an environment variable `MODEL_PATH` if it is set.
          - Otherwise, fall back to the original model path.
        - Keep the rest of the script **unchanged**.

        ### Expected Output:
        The entire Python script with only the necessary modification applied.

        ### Example Modification:
        **Before:**
        ```python
        import joblib
        model = joblib.load("model.pkl")
        ```

        **After:**
        ```python
        import os
        MODEL_PATH = os.getenv("MODEL_PATH", "model.pkl")

        import joblib
        model = joblib.load(MODEL_PATH)
        ```

        Now, apply the same transformation to the given script.

        ### Script:
        ```
        {inference_script}
        ```
        """}
    ]

    return miss_llm.invoke(messages)

In [46]:
miss_llm = MyOpenAI(model=MODEL_NAME)
# Step 2: Modify the model loading logic through an LLM without changing other parts
updated_inference_script = modify_model_loading_with_llm(inference_script)

# Step 3: Save or use the modified script
print(updated_inference_script)  # This script now correctly loads the model from ENV variable

Here is the modified Python script with the necessary changes applied to load the model from an environment variable `MODEL_PATH` if it is set, otherwise falling back to the original model path:

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np
import os

# Load the trained model
MODEL_PATH = os.getenv("MODEL_PATH", "iris_model.pkl")
model = joblib.load(MODEL_PATH)

# Create a FastAPI instance
app = FastAPI()

# Define the input data model
class IrisInput(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

# Define the prediction endpoint
@app.post("/predict")
def predict(iris_input: IrisInput):
    # Prepare the input data for prediction
    input_data = np.array([[iris_input.sepal_length, iris_input.sepal_width,
                             iris_input.petal_length, iris_input.petal_width]])
    
    # Make prediction
    try:
        prediction = model.predict(in

In [57]:
#etract the updated inference script and save it
updated_inference_script = extract_python_script(updated_inference_script)
updated_inference_script_path = '/Users/neel/Developer/deploy_wizard/iris_model_inference/inference.py'
write_to_file(updated_inference_script_path, updated_inference_script)

In [ ]:
#specify all paths
inference_script_path = '/Users/neel/Developer/deploy_wizard/iris_model_inference/inference.py'
requirements_txt_path = '/Users/neel/Developer/deploy_wizard/iris_model_inference/requirements.txt'
dockerfile_path = '/Users/neel/Developer/deploy_wizard/iris_model_inference/Dockerfile'
model_path = '/Users/neel/Developer/deploy_wizard/iris_model_inference/iris_model.pkl'
#read all files 
inference_script = file_reader(inference_script_path)
requirements_txt = file_reader(requirements_txt_path)
dockerfile = file_reader(dockerfile_path)


In [11]:
def generate_dockerfile(options, feedback=None):
    if feedback is None:
        messages = [
            {
                "role": "system",
                "content": "You are a Docker expert."
            },
            {
                "role": "user",
                "content": f'''
        You are an expert Dockerfile Generation Agent. Your task is to generate a **Dockerfile** that encapsulates a complete model inference environment.

        ### Role:
        Assist in writing a Dockerfile for a containerized model inference application using **only relative paths**.

        ### Context:
        - **Working Directory**: The Dockerfile is located inside the project directory where the model inference script, requirements file, and model file are also stored.
        - **Inference Script**:
            - Location (relative to the Dockerfile): `{options['inference_script_path']}`
            - Contents:
            ```
            {options['inference_script_content']}
            ```
        - **Requirements File**:
            - Location (relative to the Dockerfile): `{options['requirements_txt_path']}`
            - Contents:
            ```
            {options['requirements_txt_content']}
            ```
        - **Model File**:
            - Location (relative to the Dockerfile): `{options['model_path']}`

        ### Action:
        Generate a **Dockerfile** that:
        - Uses an appropriate base image (e.g., `python:3.9-slim`).
        - Sets the working directory to `/app`.
        - Installs Python dependencies from the provided requirements file.
        - **Copies all necessary files using relative paths (i.e., no absolute paths)**.
        - Ensures that the model file is placed inside `/app/model/` and creates the directory if needed.
        - Configures any necessary environment variables (e.g., `MODEL_PATH=/app/model/{options['model_path'].split('/')[-1]}`).
        - Exposes a port if the inference script serves an API (e.g., FastAPI exposing `8000`).
        - Specifies the command or entrypoint to run the inference script.
        - Follows **best practices** for caching and cleanup (e.g., using `--no-cache-dir` for `pip install`).

        ### Additional Constraints:
        - **Do not use absolute paths.**  
        - **Assume all files are within the same directory as the Dockerfile when running `docker build .`.**
        - **Ensure the COPY commands properly reflect this.**
        
        ### Expected Output:
        Provide a complete **Dockerfile** as a code block with Dockerfile syntax highlighting.
        '''
            }
        ]
    else:
        messages = feedback
    response = docker_generator_llm.invoke(messages)
    return response

In [10]:
docker_generator_llm = MyOpenAI(model=MODEL_NAME)

In [9]:
context_for_dockerfile = {
    "inference_script_path": inference_script_path,
    "inference_script_content": inference_script,
    "requirements_txt_path": requirements_txt_path,
    "requirements_txt_content": requirements_txt,
    "model_path": "/Users/neel/Developer/deploy_wizard/iris_model_inference/iris_model.pkl"
}

In [12]:
raw_dockerfile = generate_dockerfile(context_for_dockerfile)

In [13]:
def extract_dockerfile(llm_response):
    """
    Extracts the main Python script from an LLM response.

    Args:
        llm_response (str): The response text containing a Python code block.

    Returns:
        str: The extracted Dockerfile, or an empty string if no script is found.
    """
    match = re.search(r"```Dockerfile\n(.*?)\n```", llm_response, re.DOTALL)
    return match.group(1) if match else ""


In [16]:
dockerfile = extract_dockerfile(raw_dockerfile)
print(dockerfile)

# Use an appropriate base image
FROM python:3.9-slim

# Set the working directory
WORKDIR /app

# Copy the requirements file and install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Create a directory for the model
RUN mkdir -p model

# Copy the model file and inference script into the container
COPY iris_model.pkl model/
COPY inference.py .

# Set environment variable for the model path
ENV MODEL_PATH=/app/model/iris_model.pkl

# Expose the port for the FastAPI application
EXPOSE 8000

# Specify the command to run the inference script
CMD ["python", "inference.py"]


In [22]:
dockerfile_path = '/Users/neel/Developer/deploy_wizard/iris_model_inference/Dockerfile'
write_to_file(dockerfile_path, dockerfile)

In [20]:
def docker_test(messages):
    response = docker_generator_llm.invoke(messages)
    return response

## Testing the dockerized model locally

In [27]:
#local testind and deployedment
def dockerfile_testing():
    print("We will now try validating that the Dockerfile is correctly set up for building and running the model inference application.")
    print("This will involve building the Docker image locally and running a container to test the model predictions.")
    print("Let's start by building the Docker image.")
    message ="Can you provide me instructions on how to build this docker image and run it, locally?"
    while True:
        response = docker_generator_llm.invoke(message)
        print(response)
        message = input("Enter your response, if everything is working say gg: ")
        if message == 'gg':
            break


In [29]:
dockerfile_testing()

We will now try validating that the Dockerfile is correctly set up for building and running the model inference application.
This will involve building the Docker image locally and running a container to test the model predictions.
Let's start by building the Docker image.
Certainly! Here are the step-by-step instructions to build and run the Docker image locally:

### Prerequisites
1. **Docker Installed**: Ensure you have Docker installed on your machine. You can download it from [Docker's official website](https://www.docker.com/get-started).

### Instructions

1. **Navigate to the Project Directory**:
   Open a terminal and navigate to the directory where your `Dockerfile`, `requirements.txt`, `inference.py`, and `iris_model.pkl` files are located. For example:
   ```bash
   cd /Users/neel/Developer/deploy_wizard/iris_model_inference
   ```

2. **Build the Docker Image**:
   Use the following command to build the Docker image. You can name the image (e.g., `iris_model_inference`) as

# Push to dockerhub (optional)

In [66]:
msg = "Can You provide me instruction for pushing the container to dockerhub"
response = docker_generator_llm.invoke(msg)
print(response)

Certainly! Here are the step-by-step instructions for pushing your Docker container to Docker Hub:

### Prerequisites
1. **Docker Installed**: Ensure you have Docker installed on your machine.
2. **Docker Hub Account**: You need to have a Docker Hub account. If you don't have one, you can sign up at [Docker Hub](https://hub.docker.com/).

### Instructions

1. **Log in to Docker Hub**:
   Open a terminal and log in to your Docker Hub account using the following command:
   ```bash
   docker login
   ```
   You will be prompted to enter your Docker Hub username and password.

2. **Tag Your Docker Image**:
   Before pushing your image, you need to tag it with your Docker Hub username and the repository name. The format is:
   ```
   <username>/<repository>:<tag>
   ```
   For example, if your Docker Hub username is `yourusername` and you want to name your repository `iris_model_inference`, you can tag your image like this:
   ```bash
   docker tag iris_model_inference yourusername/iris_mo

## terraform for EC2 instance creation

In [103]:
terraform_generator_llm = MyOpenAI(model=MODEL_NAME)
terraform_validator_llm = MyOpenAI(model=MODEL_NAME)

In [113]:
base_main_tf = file_reader('/Users/neel/Developer/deploy_wizard/templates/aws_ec2/main.tf')
base_vars_tf = file_reader('/Users/neel/Developer/deploy_wizard/templates/aws_ec2/variables.tf')
# Define parameters in a dictionary
params = {
    "vpc_id": "vpc-0f0aea174086b6625",
    "region": "us-west-1",
    "subnet_id": "subnet-070d54662e68443ed",
    "instance_type": "t2.micro",
    "security_group_id": "sg-03aa3023dd84cf4a5",
    "key_pair_name": "neel_test",
    "ami_id": "ami-08d4f6bbae664bd41",
    "model_port": 8000,
    "model_name": "iris_model",
    "image_name": "neel26d/iris_model_inference:latest",
    "container_name": "iris_model",
    "main_tf": base_main_tf,
    "vars_tf": base_vars_tf
}
def generate_ec2_terraform_file(params,feedback=None):
    if not feedback:
        messages = [
            {"role": "system", "content": "You are an expert Terraform developer."},
            {"role": "user",
            "content": f'''
            ## Role:
            You are an **expert DevOps assistant** specializing in Terraform and AWS EC2 deployments. Your task is to generate valid Terraform files (`main.tf` and `variables.tf`) while maintaining correct syntax.

            ## Problem Statement:
            A user wants to deploy a **machine learning model** as an API on an **AWS EC2 instance using Docker**. The Terraform script should:
            - Create an **EC2 instance** inside a specified **VPC and Subnet**.
            - Configure a **Security Group** to allow **SSH, HTTP, HTTPS, and API traffic**.
            - Pull a Docker image and run it on the EC2 instance.
            - Replace placeholders with provided variable values.
            - For rules in securtiy group if no ips are specified then consider all ips 0.0.0.0/0

            ## **Terraform Templates**
            ### `main.tf`
            {params['main_tf']} 
            +/n/n
            ### `variables.tf`
            {params['vars_tf']} 

            ##Provided values:
            - **VPC ID**: {params['vpc_id']}
            - **Region**: {params['region']}
            - **Subnet ID**: {params['subnet_id']}
            - **Instance Type**: {params['instance_type']}
            - **Security Group ID**: {params['security_group_id']}
            - **Key Pair Name**: {params['key_pair_name']}
            - **AMI ID**: {params['ami_id']}
            - **Model Port**: {params['model_port']}
            - **Model Name**: {params['model_name']}   
            - **Image Name**: {params['image_name']}
            - **Container Name**: {params['container_name']}
            - Make sure these are set as default values in values.yaml

            ## Expected Output:
            - **Replace all placeholders** in `main.tf` and `variables.tf` with actual values.
            - **Maintain correct Terraform syntax**.
            - **Return two separate files (`main.tf` and `variables.tf`)**.
            - **Ensure the Terraform script is valid and deployable**.
            - **Output must be structured** as separate code blocks for `main.tf` and `variables.tf`.
            - It should be ``hcl \n ### filename`` syntax highlighted.
            '''}
        ]
    else:
        messages = feedback
    response = terraform_generator_llm.invoke(messages)
    return response

def evaluate_terraform_with_llm(terraform_code):
    """Use an LLM to evaluate the generated Terraform script and provide feedback."""
    messages = [
        {"role": "system", "content": "You are a Terraform expert and code reviewer specializing in AWS infrastructure."},
        {"role": "user", "content": f"""
        You are reviewing a Terraform script that provisions an AWS EC2 instance.

        ### Task:
        - Analyze the script for correctness, best practices, and security compliance.
        - Identify any missing components or improvements.
        - If the script is perfect, respond with 'No changes needed.'
        - If changes are required, specify what needs to be improved.
        - Make Sure all variables are set as default values in values.yaml based on the values given
        - If some value is explicitly mentioned in the script then it should be set as default value in values.yaml
        - Dont use any new variables in the script, or use values not provided in the prompt


        ### Script to review:
        ```
        {terraform_code}
        ```

        ### Expected Output:
        - Either 'No changes needed.' OR a detailed improvement plan.
        """}
    ]

    return terraform_validator_llm.invoke(messages)


def actor_critic_terraform(params):
    """
    Runs the actor-critic loop for Terraform:
    - Actor generates Terraform code.
    - Critic evaluates the script.
    - If the script is sufficient, exit early.
    - Otherwise, Actor refines the script based on feedback.
    - Max iterations: 2
    """
    feedback = None  # No feedback initially
    # Step 1: Actor generates Terraform script
    terraform_script = generate_ec2_terraform_file(params, feedback)
    for _ in range(2):  # Max iterations: 2
        # Step 2: Critic evaluates the Terraform script
        critic_feedback = evaluate_terraform_with_llm(terraform_script)

        if "no changes needed" in critic_feedback.lower():
            print("✅ Terraform script is satisfactory. Exiting early.")
            return terraform_script  # Early exit if script is sufficient

        # Step 3: Actor refines Terraform script using critic's feedback
        feedback = [
            {"role": "assistant", "content": terraform_script},
            {"role": "user", "content": f"Revise based on this feedback:\n{critic_feedback}"}
        ]

        terraform_script = generate_ec2_terraform_file(params, feedback)


    print("🔄 Max iterations reached. Returning final Terraform script.")
    return terraform_script


In [114]:
raw_terraform_script = actor_critic_terraform(params)

🔄 Max iterations reached. Returning final Terraform script.


In [115]:
print(raw_terraform_script)

Here’s a revised version of the Terraform script and `values.yaml` file based on your feedback. The changes focus on improving security, clarity, and usability.

### Revised `main.tf`
```hcl
provider "aws" {
  region = var.aws_region
}

resource "aws_security_group" "ec2_sg" {
  name_prefix = "${var.resource_prefix}-sg-"

  # Allow SSH access from a specific IP address
  ingress {
    from_port   = 22
    to_port     = 22
    protocol    = "tcp"
    cidr_blocks = [var.ssh_cidr_block] # Ensure this is set to a specific IP
  }

  # Allow HTTP access (consider restricting this as needed)
  ingress {
    from_port   = 80
    to_port     = 80
    protocol    = "tcp"
    cidr_blocks = [var.http_cidr_block] # Restrict this as needed
  }

  # Allow HTTPS access (consider restricting this as needed)
  ingress {
    from_port   = 443
    to_port     = 443
    protocol    = "tcp"
    cidr_blocks = [var.https_cidr_block] # Restrict this as needed
  }

  # Allow requests to come to the model
  ingr

In [116]:
import os
import re

def extract_and_save_hcl_files(response_str, folder="aws_terraform_files"):
    """
    Extracts HCL code blocks from the given string and saves each block
    to one or more files in the specified folder.

    :param response_str: The string containing HCL code blocks.
    :param folder: The folder where the files will be saved (default is "aws_terraform_files").
    """
    # Ensure the target folder exists.
    os.makedirs(folder, exist_ok=True)
    
    # Regular expression to capture all HCL code blocks
    pattern = r"```hcl\s*(.*?)```"
    blocks = re.findall(pattern, response_str, re.DOTALL)

    for block in blocks:
        lines = block.strip().splitlines()
        current_filename = None
        current_lines = []
        
        for line in lines:
            stripped_line = line.strip()

            # Check if the line is a header line indicating a file name
            if stripped_line.startswith("###"):  # Detect lines like "### main.tf"
                # Save the previous file before switching
                if current_filename is not None and current_lines:
                    
                    file_path = os.path.join(folder, current_filename)
                    with open(file_path, "w") as f:
                        f.write("\n".join(current_lines).rstrip())
                    print(f"Wrote file: {file_path}")

                # Extract the file name
                current_filename = stripped_line.split(" ")[1].strip()
                current_lines = []
            else:
                # If no header has been encountered yet, infer a filename
                if current_filename is None:
                    if 'resource "aws_instance"' in line:
                        current_filename = "main.tf"
                    elif 'variable' in line:
                        current_filename = "variables.tf"
                    elif 'output' in line:
                        current_filename = "outputs.tf"
                    else:
                        current_filename = "misc.tf"  # Fallback default
                current_lines.append(line)

        # After processing all lines in the block, write out the last file
        if current_filename and current_lines:
            file_path = os.path.join(folder, current_filename)
            with open(file_path, "w") as f:
                f.write("\n".join(current_lines).rstrip())
            print(f"Wrote file: {file_path}")

In [117]:
extract_and_save_hcl_files(raw_terraform_script)
print("")

Wrote file: aws_terraform_files/misc.tf
Wrote file: aws_terraform_files/variables.tf



In [118]:
msg = "Can you provde the things I will have to change before running the terraform script and instructuion on running it"
response = terraform_generator_llm.invoke(msg)
print(response)

Certainly! Below are the steps you need to take before running the Terraform script, along with instructions on how to execute it.

### Pre-Execution Changes

1. **Update `ssh_cidr_block`**:
   - In the `values.yaml` file, replace the placeholder `YOUR_IP_ADDRESS/32` with your actual public IP address. This is crucial for allowing SSH access to the EC2 instance.
   ```yaml
   ssh_cidr_block: "YOUR_ACTUAL_IP_ADDRESS/32" # Replace with your actual IP address
   ```

2. **Review and Update Other CIDR Blocks**:
   - If you want to restrict access for HTTP, HTTPS, or model access, update the `http_cidr_block`, `https_cidr_block`, and `model_cidr_block` in the `values.yaml` file as needed.

3. **Check the AMI ID**:
   - Ensure that the `ami_id` in the `values.yaml` file is valid for your selected AWS region. You can find the appropriate AMI ID in the AWS Management Console.

4. **Select the Instance Type**:
   - Review the `instance_type` in the `values.yaml` file. The default is set to `t2.

In [ ]:
#I have a terraform code, i need to ensure it can do the following - I will provide
# security group provides access to protocols and ports for all ip in the followinf format (port, protocol)
# appropriate values in the terraform are present - i will
# init script is present to install docker and other dependencies and run the docker image as I specify

In [64]:
def terraform_deployment_validator():
    msg = "Can you provide me instructions on how to deploy this terraform script?"
    response = terraform_generator_llm.invoke(msg)
    print(response)
    while True:
        msg = input("Enter your response, if everything is working say gg: ")
        if msg == 'gg':
            break
        response = terraform_generator_llm.invoke(msg)
        print(response)

## Integration testing

In [31]:
from aws_helper.ec2_deploy_wizard import DeployEC2

In [32]:
ec2_deploy = DeployEC2()

In [ ]:
ec2_deploy.get_inferece_script()

In [33]:
ec2_deploy.infernce_dir = '/Users/neel/Developer/ec2_iris'
# ec2_deploy.get_requirements_txt()

In [11]:
ec2_deploy.dockerizing_the_model()

We will now try validating that the Dockerfile is correctly set up for building and running the model inference application.
This will involve building the Docker image locally and running a container to test the model predictions.
Let's start by building the Docker image.
Certainly! Here are the step-by-step instructions to build and run the Docker image locally:

### Prerequisites
1. **Docker Installed**: Ensure you have Docker installed on your machine. You can download it from [Docker's official website](https://www.docker.com/get-started).
2. **Project Directory**: Make sure your project directory contains the following files:
   - `Dockerfile`
   - `inference.py`
   - `requirements.txt`
   - `iris_model.pkl`

### Steps to Build and Run the Docker Image

1. **Open a Terminal**: Navigate to the directory where your `Dockerfile` and other files are located.

2. **Build the Docker Image**:
   Run the following command to build the Docker image. You can replace `my-iris-model` with an

In [34]:
ec2_deploy.orchestrate_terraform_deployment()

🔄 Max iterations reached. Returning final Terraform script.


> Entering new AgentExecutor chain...

Invoking: `save_file_tool` with `{'file_path': '/Users/neel/Developer/ec2_iris/aws_terraform/main.tf', 'content': 'provider "aws" {\n  region = var.aws_region\n}\n\nresource "aws_security_group" "ec2_sg" {\n  name_prefix = var.security_group_name\n\n  # Allow SSH access from a specific IP range\n  ingress {\n    from_port   = 22\n    to_port     = 22\n    protocol    = "tcp"\n    cidr_blocks = var.allowed_ssh_cidr_blocks\n  }\n\n  # Allow HTTP access from a specific IP range\n  ingress {\n    from_port   = 80\n    to_port     = 80\n    protocol    = "tcp"\n    cidr_blocks = var.allowed_http_cidr_blocks\n  }\n\n  # Allow HTTPS access from a specific IP range\n  ingress {\n    from_port   = 443\n    to_port     = 443\n    protocol    = "tcp"\n    cidr_blocks = var.allowed_https_cidr_blocks\n  }\n\n  # Allow application-specific port access from a specific IP range\n  ingress {\n    from_p